In [1]:
import os
import sys
import transformers
import torch
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import TFAutoModelForSeq2SeqLM
from transformers import DataCollatorForSeq2Seq
from transformers import AdamWeightDecay
from transformers import BertTokenizerFast
import tensorflow as tf

from transformers import AutoModelForTokenClassification,AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments, DataCollator

c:\Users\satya\anaconda3\envs\transformers\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from datasets import load_dataset
csv='C:/Users/satya/Downloads/Resume.csv/Resume.csv'
resume_ds=load_dataset('csv',data_files=csv)

In [3]:
resume_ds

DatasetDict({
    train: Dataset({
        features: ['ID', 'Resume_str', 'Resume_html', 'Category'],
        num_rows: 2484
    })
})

In [20]:

model=AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=24)
tokenizer=BertTokenizerFast.from_pretrained('bert-base-uncased')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
example_text=resume_ds['train'][0]
example_text

{'ID': 16852973,
 'Resume_str': "         HR ADMINISTRATOR/MARKETING ASSOCIATE\n\nHR ADMINISTRATOR       Summary     Dedicated Customer Service Manager with 15+ years of experience in Hospitality and Customer Service Management.   Respected builder and leader of customer-focused teams; strives to instill a shared, enthusiastic commitment to customer service.         Highlights         Focused on customer satisfaction  Team management  Marketing savvy  Conflict resolution techniques     Training and development  Skilled multi-tasker  Client relations specialist           Accomplishments      Missouri DOT Supervisor Training Certification  Certified by IHG in Customer Loyalty and Marketing by Segment   Hilton Worldwide General Manager Training Certification  Accomplished Trainer for cross server hospitality systems such as    Hilton OnQ  ,   Micros    Opera PMS   , Fidelio    OPERA    Reservation System (ORS) ,   Holidex    Completed courses and seminars in customer service, sales strate

In [22]:
tokenized_input=tokenizer(example_text['Resume_str'], is_split_into_words=False,truncation=True)

In [23]:
tokenized_input

{'input_ids': [101, 17850, 8911, 1013, 5821, 5482, 17850, 8911, 12654, 4056, 8013, 2326, 3208, 2007, 2321, 1009, 2086, 1997, 3325, 1999, 15961, 1998, 8013, 2326, 2968, 1012, 9768, 12508, 1998, 3003, 1997, 8013, 1011, 4208, 2780, 1025, 29453, 2015, 2000, 16021, 28345, 1037, 4207, 1010, 14727, 8426, 2000, 8013, 2326, 1012, 11637, 4208, 2006, 8013, 9967, 2136, 2968, 5821, 7842, 2615, 10736, 4736, 5813, 5461, 2731, 1998, 2458, 10571, 4800, 1011, 4708, 2121, 7396, 4262, 8325, 17571, 5284, 11089, 12366, 2731, 10618, 7378, 2011, 1045, 25619, 1999, 8013, 9721, 1998, 5821, 2011, 6903, 15481, 4969, 2236, 3208, 2731, 10618, 8885, 10365, 2005, 2892, 8241, 15961, 3001, 2107, 2004, 15481, 2006, 4160, 1010, 12702, 2015, 3850, 7610, 2015, 1010, 26000, 12798, 3850, 11079, 2291, 1006, 2030, 2015, 1007, 1010, 7570, 24198, 2595, 2949, 5352, 1998, 17239, 1999, 8013, 2326, 1010, 4341, 9942, 1010, 12612, 2491, 1010, 3279, 9740, 1010, 3808, 1010, 2051, 2968, 1010, 4105, 1998, 2836, 7667, 1012, 3325, 17850, 89

In [24]:
tokenz=tokenizer.convert_ids_to_tokens(tokenized_input['input_ids'])

array(['ACCOUNTANT', 'ADVOCATE', 'AGRICULTURE', 'APPAREL', 'ARTS',
       'AUTOMOBILE', 'AVIATION', 'BANKING', 'BPO', 'BUSINESS-DEVELOPMENT',
       'CHEF', 'CONSTRUCTION', 'CONSULTANT', 'DESIGNER', 'DIGITAL-MEDIA',
       'ENGINEERING', 'FINANCE', 'FITNESS', 'HEALTHCARE', 'HR',
       'INFORMATION-TECHNOLOGY', 'PUBLIC-RELATIONS', 'SALES', 'TEACHER'],
      dtype='<U22')

In [37]:
LE.get_params

<bound method BaseEstimator.get_params of LabelEncoder()>

In [25]:
from sklearn.preprocessing import LabelEncoder
LE=LabelEncoder()
LE.fit(resume_ds['train']['Category'])
def preprocess(data):
  input_encoding=tokenizer(data['Resume_str'], is_split_into_words=False,truncation=True, padding=True, return_tensors='pt')

  label_encoding = LE.transform(data['Category'])
  #print(label_encoding)
  input_encoding['labels']=label_encoding
  return input_encoding

In [26]:
import tensorflow as tf
mapped_data=resume_ds.map(preprocess, batched=True)

In [27]:
import numpy as np 
np.unique(mapped_data['train']['labels'])

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23])

In [28]:
from transformers import DataCollatorForTokenClassification, DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [29]:
args=TrainingArguments(
    "test-ner",evaluation_strategy='epoch',
    learning_rate=0.0005,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01
)
trainer=Trainer(
    model, 
    args,
    train_dataset=mapped_data['train'],
    eval_dataset=mapped_data['train'],
    data_collator=data_collator,
    tokenizer=tokenizer,
 

)

  0%|          | 0/156 [02:50<?, ?it/s]


In [2]:
trainer.train()

NameError: name 'trainer' is not defined

In [1]:
model.save_pretrained('resume_parser')

NameError: name 'model' is not defined

In [31]:
from transformers import pipeline

In [32]:
nlp_pipeline=pipeline('text-classification',model=model, tokenizer=tokenizer)

In [33]:
nlp_pipeline('2 years experience in gcp')

[{'label': 'LABEL_15', 'score': 0.059371527284383774}]

In [38]:
classes=LE.classes_
classes

array(['ACCOUNTANT', 'ADVOCATE', 'AGRICULTURE', 'APPAREL', 'ARTS',
       'AUTOMOBILE', 'AVIATION', 'BANKING', 'BPO', 'BUSINESS-DEVELOPMENT',
       'CHEF', 'CONSTRUCTION', 'CONSULTANT', 'DESIGNER', 'DIGITAL-MEDIA',
       'ENGINEERING', 'FINANCE', 'FITNESS', 'HEALTHCARE', 'HR',
       'INFORMATION-TECHNOLOGY', 'PUBLIC-RELATIONS', 'SALES', 'TEACHER'],
      dtype='<U22')